In [1]:

import numpy as np
import matplotlib.pyplot as plt
import os, sys
import pickle as pk
import time
import pandas as pd

import utils as ut
import time
import math
import copy
from itertools import chain
from GPyOpt.methods import BayesianOptimization
import scipy
from functools import partial

In [2]:
# constants
# rhino uv nurbs +1 (even numbers required?
u = 8 #u = number of points
v = 5 #v = number of platforms
#f_experiment = '200616'
curr_dir_repo = os.getcwd().split('/src/python')[0]
dataset_dir = curr_dir_repo + '/data/' #+ f_experiment
results_dir = curr_dir_repo + '/results/' #+ f_experiment

fn_attr_ranges = 'ranges_attrib.pkl'

###
# THE NOTATION IS
# W for geometries: the points, either on xyz or polar, for each of the platforms
# X for desing attributes: final desired characteristics, like rain occlusion,
# sun occlusion, surface, etc.

flag_polar = True

fname = os.path.join(dataset_dir, 'data_labels_all_200706.pkl')
list_all_dicts = pk.load(open(fname, 'rb'))

# %%

flag_GH_xyz_polar = 0
flag_out_xyz_polar = 0
if flag_polar:
    flag_GH_xyz_polar = 1
    flag_out_xyz_polar = 1

w_train_red, bottom_top_heights = ut.gh_to_script(list_all_dicts, v, u, flag_GH_xyz_polar=flag_GH_xyz_polar,
                                                  flag_out_xyz_polar=flag_out_xyz_polar)

# %%
# Import and preprocess y_train: design attributes


# Attributes to use
list_attrib = ['occlusion_rain', 'occlusion_sun',
               'outline_lengths_0', 'outline_lengths_1', 'outline_lengths_2', 'outline_lengths_3', 'outline_lengths_4',
               'surface_areas_0', 'surface_areas_1', 'surface_areas_2', 'surface_areas_3', 'surface_areas_4']
x_train = ut.ghlabels_to_script(list_all_dicts, list_attrib, flag_all_df=True)

all_attributes = list(x_train.columns)
print(all_attributes)


num_attr = len(list_attrib) # number of attributes selected for training

#print("Attributes",list_attrib)

x_train_red = np.asarray(x_train[list_attrib])

w_nans, ind_not_nanw = ut.check_rows_nan(w_train_red)
x_nans, ind_not_nanx = ut.check_rows_nan(x_train_red)

if len(np.union1d(w_nans, x_nans)):
    ind_not_nan = np.intersect1d(ind_not_nanw, ind_not_nanx)
    print('Reducing samples from {} to {}'.format(len(w_train_red), len(ind_not_nan)))
    w_train_red = w_train_red[ind_not_nan, :]
    x_train_red = x_train_red[ind_not_nan, :]
else:
    print('No nans! ¯\_(ツ)_/¯')


data_dir_loop = '/Users/pouya/vegetale_bayopt'
str_save = 'It1'
flag_save_files = True
max_time = 3600
output_attrib = ['occlusion_rain', 'occlusion_sun']


['cantilever_max', 'occlusion_sun', 'outline_lengths_0', 'outline_lengths_1', 'outline_lengths_2', 'outline_lengths_3', 'outline_lengths_4', 'surface_area_total', 'buildable', 'surface_areas_0', 'surface_areas_1', 'surface_areas_2', 'surface_areas_3', 'surface_areas_4', 'outline_length_total', 'supports_count', 'outlines_bleps', 'occlusion_rain', 'parcel_intersection', 'supports_problems', 'columns_max_length']
No nans! ¯\_(ツ)_/¯


In [3]:
xy_ind = list(chain.from_iterable((i, i + 1) for i in range(10, 50, 10)))
h_ind = [50, 52, 51]
all_inds = xy_ind+h_ind


true_vals = w_train_red[:,all_inds]
w_train_red[:,xy_ind] = 0 #np.random.normal(size=(w_train_red.shape[0],len(xy_ind)))
w_train_red[:,50] = 5.5
w_train_red[:,51] = 6.5
w_train_red[:,52] = 7.5


In [73]:
def GH_black_box_minimize(index,all_ind ,**kwargs):
    str_save = "Iter" + str(index)
    #print(kwargs)
    print(kwargs)
    params = [kwargs["x1"],kwargs["y1"],kwargs["x2"],kwargs["y2"],kwargs["x3"],kwargs["y3"],
              kwargs["x4"],kwargs["y4"],kwargs["h1"],kwargs["h2"],kwargs["h3"]]
    w_train_red[index,all_ind] = params

    list_dicts = ut.polar_to_ghpolar(w_train_red, bottom_top_heights,
                                     v, u, flag_create_dict=True,
                                     x_in=x_train_red, list_attrib=list_attrib,
                                     flag_save_files=flag_save_files, str_save=str_save)

    list_dicts = list_dicts[:index+1]
    f_name = os.path.join(data_dir_loop, 'tmp_fromscript.pkl')
    pk.dump(list_dicts, open(f_name, 'wb'), protocol=2)
    if len(str_save):
        f_name = os.path.join(data_dir_loop, 'saved/tmp_fromscript_{}.pkl'.format(str_save))
        pk.dump(list_dicts, open(f_name, 'wb'), protocol=2)

    t_st = time.time()
    f_name_out = os.path.join(data_dir_loop, 'tmp_fromgh.pkl')
    print('Waiting for file...')
    dict_polar = ut.wait_gh_x(f_name_out, max_time)
    t_end = time.time() - t_st
    print('Finished the loop on GH for %d samples in total time of %.2f' % (len(list_dicts), t_end))
    x_gh = ut.ghlabels_to_script(dict_polar, list_attrib, flag_all_df=True)[output_attrib]
    xgh = np.asarray(x_gh.values)
    rain_err= (xgh[:,0] - x_train_red[index,0])**2
    sun_err = (xgh[:,1] - x_train_red[index,1])**2
    err = math.sqrt(np.sum(rain_err)+ np.sum(sun_err))/2

  
    print("ERROR {} in Garden {}".format(err,index))

    return err

In [74]:
from bayes_opt import BayesianOptimization, UtilityFunction



lower_xy = -3
upper_xy = 3
mu = 0
sigma = 1.0
lower_h = 4.5
upper_h = 19



bounds = {'x1': (lower_xy,upper_xy),'y1': (lower_xy,upper_xy),'x2': (lower_xy,upper_xy),
          'y2': (lower_xy,upper_xy),
          'x3': (lower_xy,upper_xy),
          'y3': (lower_xy,upper_xy),
          'x4': (lower_xy,upper_xy),
          'y4': (lower_xy,upper_xy),
          'h1': (lower_h,upper_h),
          'h2': (lower_h,upper_h),
          'h3': (lower_h,upper_h)}



          

In [78]:
optimizer = BayesianOptimization(
    f=None,
    pbounds=bounds,
    verbose=2,
    random_state=1,
)

util = UtilityFunction(kind="poi", kappa=1, xi=0.0)

In [79]:
#partial(GH_black_box_minimize(10,all_ind))
for _ in range(5):
    print(util)
    next_point = optimizer.suggest(util)
    target = GH_black_box_minimize(10,all_inds,**next_point)
    optimizer.register(params=next_point, target=target)
    print(target, next_point)
print(optimizer.max)

{'h1': 10.546819068187322, 'h2': 14.944705154911292, 'h3': 4.5016584348515005, 'x1': -1.1860045642089614, 'x2': -2.119464655097322, 'x3': -2.445968431387213, 'x4': -1.8824387317339746, 'y1': -0.9266356377417138, 'y2': -0.6193951546159804, 'y3': 0.23290040402014167, 'y4': -0.48483291358023095}
Waiting for file...
Finished the loop on GH for 11 samples in total time of 2.01
ERROR 42.049370351530705 in Garden 10
42.049370351530705 {'h1': 10.546819068187322, 'h2': 14.944705154911292, 'h3': 4.5016584348515005, 'x1': -1.1860045642089614, 'x2': -2.119464655097322, 'x3': -2.445968431387213, 'x4': -1.8824387317339746, 'y1': -0.9266356377417138, 'y2': -0.6193951546159804, 'y3': 0.23290040402014167, 'y4': -0.48483291358023095}


ValueError: array must not contain infs or NaNs